In [1]:
#for referece

# Fs = 8000
# f = 5
# sample = 8000
# x = np.arange(sample)
# y = np.sin(2 * np.pi * f * x / Fs)
# plt.plot(x, y)
# plt.xlabel('sample(n)')
# plt.ylabel('voltage(V)')
# plt.show()

In [2]:
# #create data points::

# seq_len = 31
# def create_time_series(seq_len):
#   freq = (np.random.random()*0.5) + 0.1  # 0.1 to 0.6
#   ampl = np.random.random() + 0.5  # 0.5 to 1.5
#   x = np.sin(np.arange(0,seq_len) * freq) * ampl
#   return x

## tflearn rnn experiment
use with tensorlab enviorment

tasks:

    [ ] create data set for training
        [ ] sin
        [ ] sales tpv day
        [ ] 30 numerical sequences with sequntial teste
        [ ] 50 numerical sequences with shuffle test
    
    [] plot sin

    

In [3]:
import numpy as np
import tensorflow as tf
import tflearn
import matplotlib.pyplot as plt
from tflearn.data_utils import shuffle

%matplotlib inline

In [4]:
#create trainX and train Y

np_train_exp_X = np.array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32], [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36], [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37], [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42], [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43], [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46], [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47], [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48], [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51], [23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52], [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53], [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54], [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55], [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56], [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57], [29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]], np.int32)
#print(np_train_exp_X)
np_train_exp_Y = np.arange(30, 60)
print(np_train_exp_Y)

[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54
 55 56 57 58 59]


In [5]:
#reshape trainX
np_train_exp_X = np.reshape(np_train_exp_X, (30, 30,1))/10

#reshape trainY
np_train_exp_Y = np.reshape(np_train_exp_Y, (30,1))/10
np_train_exp_Y.shape
#np_train_exp_X.shape

(30, 1)

In [6]:
#create TestX and TestY

np_test_exp_X = np.array([[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], [31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60], [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61], [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62], [34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63], [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64], [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65], [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66], [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67], [39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]], np.int32) 
np_test_exp_X = np.reshape(np_test_exp_X, (10, 30,1))/10

#print(np_test_exp_X)
np_test_exp_Y = np.arange(60,70)
np_test_exp_Y = np.reshape(np_test_exp_Y, (10,1))/10


In [7]:
#np_test_exp_X

In [8]:
# x = np.array ([0,]*n)
# x = np.array ([])

In [9]:
# Define the neural network
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    #### Your code ####
    # Include the input layer, hidden layer(s), and set how you want to train the model
    
    # input layer
    
    net = tflearn.input_data(shape=[None, 30,1])
    #net = tflearn.embedding(net, input_dim=1, output_dim=60)
    # net = tflearn.lstm(net, n_units=30, return_seq=True)
    net = tflearn.lstm(net, n_units=64, return_seq=False)
    net = tflearn.fully_connected(net, 256, activation='linear')
    net = tflearn.fully_connected(net, 128, activation='linear')
    net = tflearn.fully_connected(net, 64, activation='linear')
    net = tflearn.fully_connected(net, 1, activation='linear')
    
    #Output Layer
    net = tflearn.regression(net, optimizer='adam', loss='mean_square')
    
    # This model assumes that your network is named "net"  
    #model = tflearn.DNN(net, clip_gradients=0.0, tensorboard_verbose=0)
    model = tflearn.DNN(net, clip_gradients=5.0)
    #model = tflearn.DNN(net)
    return model

In [10]:
model = build_model()

In [11]:
# Training
model.fit(np_train_exp_X, np_train_exp_Y, n_epoch=50, validation_set=0.1, show_metric=True, batch_size=10, shuffle=True)
#model.fit(train_features, train_next_point, n_epoch=1, validation_set=0.1, show_metric=True, batch_size=1)


Training Step: 149  | total loss: 0.01304 | time: 0.032s
| Adam | epoch: 050 | loss: 0.01304 - binary_acc: 0.0000 -- iter: 20/27
Training Step: 150  | total loss: 0.01176 | time: 1.055s
| Adam | epoch: 050 | loss: 0.01176 - binary_acc: 0.0000 | val_loss: 0.00319 - val_acc: 0.0000 -- iter: 27/27
--


In [12]:
# Compare the labels that our model predicts with the actual labels

# Find the indices of the most confident prediction for each item. That tells us the predicted digit for that sample.
predictions = np.array(model.predict(np_test_exp_X))

# Calculate the accuracy, which is the percentage of times the predicated labels matched the actual labels
actual = np_test_exp_Y
test_accuracy = np.mean(np.abs(predictions - actual), axis=0)

# Print out the result
print("Test accuracy: ", test_accuracy)

Test accuracy:  [ 0.08713965]


In [28]:
np.around(predictions, decimals=2)

array([[ 6.01000023],
       [ 6.09000015],
       [ 6.17999983],
       [ 6.26000023],
       [ 6.34000015],
       [ 6.40999985],
       [ 6.48999977],
       [ 6.55999994],
       [ 6.61999989],
       [ 6.69000006]], dtype=float32)

In [29]:
actual

array([[ 6. ],
       [ 6.1],
       [ 6.2],
       [ 6.3],
       [ 6.4],
       [ 6.5],
       [ 6.6],
       [ 6.7],
       [ 6.8],
       [ 6.9]])

In [30]:
#experiments


In [31]:
seq_data = np.arange(0, 100)
seq_data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [32]:
seq_len = 50
ini = 0

In [33]:

component = np.array(seq_data[ini : seq_len])

for i in range(49):
    ini = ini +1
    seq_len = seq_len + 1
    component = np.append( component, np.array(seq_data[ini : seq_len]))

In [34]:
component

array([ 0,  1,  2, ..., 96, 97, 98])

In [35]:
# for i in component:
#     print (i)

In [36]:
component = np.reshape(component, (50,50))
shuffle = component
np.random.shuffle(shuffle)
shuffle

array([[33, 34, 35, ..., 80, 81, 82],
       [48, 49, 50, ..., 95, 96, 97],
       [34, 35, 36, ..., 81, 82, 83],
       ..., 
       [28, 29, 30, ..., 75, 76, 77],
       [13, 14, 15, ..., 60, 61, 62],
       [38, 39, 40, ..., 85, 86, 87]])

In [37]:
test_fraction = 0.9
records = len(component)

train_X, test_X = shuffle[:int(records*test_fraction)], shuffle[int(records*test_fraction):]

In [49]:
records

50

In [38]:
#train_X

In [39]:
#test_X

In [40]:
train_Y = np.array(train_X[0][49:]+1)
for i in train_X[1:]:
    train_Y = np.append( train_Y, i[49:] +1)

In [41]:
test_Y = np.array(test_X[0][49:]+1)
for i in test_X[1:]:
    test_Y = np.append( test_Y, i[49:] +1)

In [42]:
train_X = np.reshape(train_X, (45, 50,1))/10
train_Y = np.reshape(train_Y, (45,1))/10
test_X = np.reshape(test_X, (5, 50,1))/10
test_Y = np.reshape(test_Y, (5,1))/10

In [43]:
# Define the neural network
def build_model2():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    # input layer
    net = tflearn.input_data(shape=[None, 50,1])
    
    # hidden layers
    # net = tflearn.lstm(net, n_units=30, return_seq=True)
    net = tflearn.lstm(net, n_units=64, return_seq=False)
    net = tflearn.fully_connected(net, 256, activation='linear')
    net = tflearn.fully_connected(net, 128, activation='linear')
    #net = tflearn.fully_connected(net, 64, activation='linear')
    net = tflearn.fully_connected(net, 1, activation='linear')
    
    #Output Layer
    net = tflearn.regression(net, optimizer='adam', loss='mean_square')
    
    # This model assumes that your network is named "net"  
    #model = tflearn.DNN(net, clip_gradients=0.0, tensorboard_verbose=0)
    model = tflearn.DNN(net, clip_gradients=5.0)
    #model = tflearn.DNN(net)
    return model

In [44]:
model2 = build_model2()

In [45]:
# Training
model2.fit(train_X, train_Y, n_epoch=50, validation_set=0.1, show_metric=True, batch_size=10, shuffle=True)
#model.fit(train_features, train_next_point, n_epoch=1, validation_set=0.1, show_metric=True, batch_size=1)

Training Step: 199  | total loss: 0.00451 | time: 0.113s
| Adam | epoch: 050 | loss: 0.00451 - binary_acc: 0.0000 -- iter: 30/40
Training Step: 200  | total loss: 0.00413 | time: 1.156s
| Adam | epoch: 050 | loss: 0.00413 - binary_acc: 0.0000 | val_loss: 0.00039 - val_acc: 0.0000 -- iter: 40/40
--


In [46]:
# Compare the labels that our model predicts with the actual labels

# Find the indices of the most confident prediction for each item. That tells us the predicted digit for that sample.
predictions = np.array(model2.predict(test_X))

# Calculate the accuracy, which is the percentage of times the predicated labels matched the actual labels
actual = test_Y
test_accuracy = np.mean(np.abs(predictions - actual), axis=0)

# Print out the result
print("Test accuracy: ", test_accuracy)

Test accuracy:  [ 0.01966362]


In [47]:
predictions

array([[ 6.47393274],
       [ 5.19376945],
       [ 7.78539133],
       [ 6.27029324],
       [ 8.82170486]], dtype=float32)

In [48]:
actual

array([[ 6.5],
       [ 5.2],
       [ 7.8],
       [ 6.3],
       [ 8.8]])